# Gradient descent y Tensorboard

## Configuración y carga de datos

In [1]:
import datetime, os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math

if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()

import tensorflow_probability as tfp


tb_logs_dir = "./logs/"
os.makedirs(tb_logs_dir, exist_ok=True)

%load_ext tensorboard
# %tensorboard --logdir {tb_logs_dir}

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
all_data = np.load('data/proyecto_training_data.npy')

training_y = all_data[:,0].reshape(-1,1) / 1000 # precio de venta en miles de dólares
training_x = all_data[:,1].reshape(-1,1)
n_sample = len(training_y)


## Formulación de hipótesis

Verificamos que las variables presentan un coeficiente de correlación positivo y considerablemente alto. Esto nos indica que un modelo de regresión lineal podría ajustarse a los datos.

In [3]:

corr = tfp.experimental.substrates.numpy.stats.correlation(np.copy(training_x), np.copy(training_y))
print(corr)

[[0.7909816]]


## Definicion de grafo

In [17]:
class Model:
    def __init__(self, training_x, training_y, learning_rate):
        with tf.name_scope("datos_de_entrenamiento"):
            self.x = tf.concat([training_x, tf.ones([training_x.shape[0], 1], dtype=tf.float64)], axis = 1)
            print(self.x)
            self.y = training_y
            self.learning_rate = learning_rate
        with tf.name_scope("pesos"):
            self.weights = tf.Variable(name="Weights", initial_value=tf.zeros((2,1),tf.float64))
        with tf.name_scope("predecir_y_hat"):
            y_hat = tf.matmul(self.x, self.weights)
        with tf.name_scope("calcular_error"):
            self.error = (1/2 * tf.reduce_mean(tf.math.square(self.y - y_hat)))
        with tf.name_scope("calcular_gradientes"):
            gradients = tf.gradients(self.error, self.weights)
        with tf.name_scope("actualizar_pesos"):
            adjustment = tf.scalar_mul(-self.learning_rate, gradients[0])
            self.update = tf.assign(self.weights, (tf.add(self.weights, adjustment)))    
        
    def epoch(self):
        return self.update
    

### Diagrama del grafo
![image.png](img/graph.png)

## Entrenamiento

In [5]:
def train(learning_rate, epochs, print_rate=200):
    print("lr=", str(learning_rate))
    tf.reset_default_graph()

    placeholder_x = tf.placeholder(tf.float64,[n_sample, 1],"overall_qual")
    placeholder_y = tf.placeholder(tf.float64,[n_sample, 1],"sale_price")

    with tf.Session() as session:    
        feed_dict = {placeholder_x: training_x, placeholder_y: training_y}
        model = Model(placeholder_x, placeholder_y, learning_rate)
        error_summary = tf.summary.scalar(name='Error', tensor=model.error)

        timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')
        log_name = tb_logs_dir + timestamp + "_lr=" + str(learning_rate) + "_epochs=" + str(epochs)
        writer = tf.summary.FileWriter(log_name, session.graph)

        session.run(tf.global_variables_initializer())
        
        for epoch in range(1, epochs + 1):
            session.run(model.epoch(), feed_dict=feed_dict)
            writer.add_summary(session.run(error_summary, feed_dict=feed_dict), epoch)
            
            if epoch % print_rate == 0 or epoch == epochs:
                error = session.run(model.error, feed_dict=feed_dict)
                print("epoch:" + str(epoch) + " error: " + str(error))
                if math.isinf(error):
                    print("terminando por divergencia")
                    break
        writer.close()
    session.close()

Vamos a utilizar varios valores para el learning_rate hasta encontrar el mejor con la ayuda de tensorboard. Luego intentaremos encontrar una cantidad de iteraciones pequeña pero suficiente para el resultado esperado.

In [18]:
epochs = 1000
for learning_rate in [0.1, 0.02, 0.01, 0.002, 0.001]:
    train(learning_rate, epochs)
    print()

lr= 0.1
Tensor("datos_de_entrenamiento/concat:0", shape=(1460, 2), dtype=float64)
epoch:200 error: 3.045890471906833e+195
epoch:400 error: inf
terminando por divergencia

lr= 0.02
Tensor("datos_de_entrenamiento/concat:0", shape=(1460, 2), dtype=float64)
epoch:200 error: 1350.0802433327196
epoch:400 error: 1296.2465977078166
epoch:600 error: 1259.4992627900178
epoch:800 error: 1234.4151985791357
epoch:1000 error: 1217.29258876966

lr= 0.01
Tensor("datos_de_entrenamiento/concat:0", shape=(1460, 2), dtype=float64)
epoch:200 error: 1385.7691204673454
epoch:400 error: 1350.0956996448563
epoch:600 error: 1320.620919644425
epoch:800 error: 1296.2676999070509
epoch:1000 error: 1276.1461142135142

lr= 0.002
Tensor("datos_de_entrenamiento/concat:0", shape=(1460, 2), dtype=float64)
epoch:200 error: 1419.6398569870832
epoch:400 error: 1410.6833858114394
epoch:600 error: 1402.0623140588518
epoch:800 error: 1393.7640817856502
epoch:1000 error: 1385.77659938947

lr= 0.001
Tensor("datos_de_entrenamien

El mejor valor de lr de los experimentos anteriores parece ser 0.02, buscaremos alrededor de ese valor a ver si encontramos uno que reduzca el error más rápido.


In [73]:
for learning_rate in [0.05, 0.04, 0.03, 0.025]:
    train(learning_rate, epochs)
    print()

lr= 0.05
epoch:200 error: 67923.49375770157
epoch:400 error: 246751.31535903833
epoch:600 error: 905760.2772557576
epoch:800 error: 3333672.6278908486
epoch:1000 error: 12278309.791716497

lr= 0.04
epoch:200 error: 1296.2043645324281
epoch:400 error: 1234.3758482265991
epoch:600 error: 1205.5770417331494
epoch:800 error: 1192.162984514879
epoch:1000 error: 1185.9149154813251

lr= 0.03
epoch:200 error: 1320.5825991040456
epoch:400 error: 1259.4776471274602
epoch:600 error: 1225.0208351918222
epoch:800 error: 1205.5907914660236
epoch:1000 error: 1194.6342758170529

lr= 0.025
epoch:200 error: 1334.6292623051604
epoch:400 error: 1276.1134201203938
epoch:600 error: 1239.808642185996
epoch:800 error: 1217.2841966400495
epoch:1000 error: 1203.309434197787



Vemos que un valor de 0.5 comienza a diverger, mientras que 0.04 no diverge y es el valor que más rápido ha reducido el error. buscando valores entre 0.05 y 0.04 encontramos el que parece ser el ideal: 0.0498

In [74]:
for learning_rate in [0.0499, 0.0498, 0.0497, 0.0496]:
    train(learning_rate, 2000, print_rate=400)
    print()

lr= 0.0499
epoch:400 error: 11214.003731016739
epoch:800 error: 6709.921674005602
epoch:1200 error: 4233.737957089572
epoch:1600 error: 2867.3257489779458
epoch:2000 error: 2112.5488823938686

lr= 0.0498
epoch:400 error: 1619.3239904480738
epoch:800 error: 1194.9179484147107
epoch:1200 error: 1181.4896319198906
epoch:1600 error: 1180.5945136255332
epoch:2000 error: 1180.485616460357

lr= 0.0497
epoch:400 error: 1233.6084169762803
epoch:800 error: 1186.050368049776
epoch:1200 error: 1181.3009521433075
epoch:1600 error: 1180.592006936388
epoch:2000 error: 1180.4858727317653

lr= 0.0496
epoch:400 error: 1218.433584718377
epoch:800 error: 1186.079115242167
epoch:1200 error: 1181.310563634387
epoch:1600 error: 1180.5939316618885
epoch:2000 error: 1180.4862336132403



Ahora buscamos el número óptimo de iteraciones:

In [75]:
learning_rate = 0.0498
train(learning_rate, 5_000, print_rate=500)

lr= 0.0498
epoch:500 error: 1358.608913083541
epoch:1000 error: 1183.9314695668368
epoch:1500 error: 1180.6763977712426
epoch:2000 error: 1180.485616460357
epoch:2500 error: 1180.4688857151407
epoch:3000 error: 1180.4673432626587
epoch:3500 error: 1180.4672003780743
epoch:4000 error: 1180.4671871361313
epoch:4500 error: 1180.4671859088735
epoch:5000 error: 1180.4671857951307


Más allá de las 4000 iteraciones, la ganancia es marginalmente más pequeña cada vez. Vemos que a partir de las 1500 iteraciones, el error se estabiliza alrededor de 1180. Más allá de las 2500 iteraciones, la mejora es marginalmente más pequeña cada vez. 

#### lr=0.1 y lr=.0.05hacen que el error diverja, mientras que lr=0.49 lo hace converger
![img/01.jpg](img/01.png)


#### Al analizar las primeras 80 iteraciones, vemos que un lr más bajo causa un error inicial más bajo, pero también una velocidad de convergencia más lenta, como podemos ver al analizar más iteraciones
![image.png](img/02.png)

## Conclusiones
Al evaluar las gráficas y los datos numéricos por más iteraciones, notamos que aunque el error inicial es mucho más pequeño con lr=0.01, el valor de lr=0.0498 se acerca al valor límite a una velocidad mucho más rápida, por lo que tiene la ventaja de requerir menos iteraciones para entrenarse, siempre y cuando le de tiempo de compensar el error alto en las primeras 600 o 700 iteraciones aproximadamente.

Después de unas 4000 iteraciones, el lr=0.01 llega a acercarse bastante al otro, pero incluso tras 10,000 iteraciones, el learning rate más alto todavía logra un valor numérico más bajo del error por algunos decimales. Lo cual posiblemente ya no sea significativo para la aplicación real, aunque al ser una cantidad en dólares al cuadrado, puede ser que sí interese llegar al valor más bajo posible. En ese caso, se seleccionaría el experimento con **lr=0.498 y 4000 a 5000 iteraciones** como el mejor.

![image.png](img/03.png)